In [1]:
# the original wind file from copernicus has wind for every cell of the map
import numpy as np
import netCDF4 as nc
import json
import png #pip install pypng
import math
import pandas as pd


In [49]:
# get the wind file and extract horiz and vert vectors
copern_10m = nc.Dataset("..//data//wind_10m.nc")
print(copern_10m.variables)
#bathy1 = grid1.variables['Bathymetry'][:, :]
X1 = copern_10m.variables['longitude'][:]
Y1 = copern_10m.variables['latitude'][:]
u10 = copern_10m.variables['u10'][:,:,:,:]
v10 = copern_10m.variables['v10'][:,:,:,:]
# note time in hours since 1900-01-01 00:00:00 (cumulative from start of data extract (1981))
#time_var = copern_10m.variables['time']
# This will convert number of days since 1900-01-01 (i.e. the units of time) to python datetime objects
time_convert = nc.num2date(copern_10m.variables['time'][:], copern_10m.variables['time'].units, copern_10m.variables['time'].calendar)

{'longitude': <class 'netCDF4._netCDF4.Variable'>
float32 longitude(longitude)
    units: degrees_east
    long_name: longitude
unlimited dimensions: 
current shape = (25,)
filling on, default _FillValue of 9.969209968386869e+36 used, 'latitude': <class 'netCDF4._netCDF4.Variable'>
float32 latitude(latitude)
    units: degrees_north
    long_name: latitude
unlimited dimensions: 
current shape = (19,)
filling on, default _FillValue of 9.969209968386869e+36 used, 'expver': <class 'netCDF4._netCDF4.Variable'>
int32 expver(expver)
    long_name: expver
unlimited dimensions: 
current shape = (2,)
filling on, default _FillValue of -2147483647 used, 'time': <class 'netCDF4._netCDF4.Variable'>
int32 time(time)
    units: hours since 1900-01-01 00:00:00.0
    long_name: time
    calendar: gregorian
unlimited dimensions: 
current shape = (486,)
filling on, default _FillValue of -2147483647 used, 'u10': <class 'netCDF4._netCDF4.Variable'>
int16 u10(time, expver, latitude, longitude)
    scale_fac

In [52]:
time_convert[12]

datetime.datetime(1981, 1, 1, 0, 0)

In [295]:
# original script by MapBox / Agafonkin downloads 'grib' .anl files and produces a json file  
# the json file is then used to create a PNG (RBGA format)
# https://github.com/mapbox/webgl-wind/blob/master/data/prepare.js

# Vladimir Agafonkin during interpolation and shading makes use of RBGA png (4 channel)  
# only RB are used for vel vectors but during createtexture in wind-gl.js the spare alpha channel is used
# see his methods summary: https://blog.mapbox.com/how-i-built-a-wind-map-with-webgl-b63022b5537f
timestep = 364 # e.g, may 1980 = 4, may 1981 = 16 
datefilename = "2010-05"

# this can be achieved for netcdf copernicus file using png library

# https://pypng.readthedocs.io/en/latest/png.html#colour-models

#example writes a line of pixels vertically
#png.from_array([[255,0,0,255],[0,255,255,0],[255,0,0,255],[255,0,0,255]], "RGBA").save("small_smiley.png")

# get width and height of array
#print(v10.shape[3])
#print(v10[2,0,:,:])

# rescale the u and v values (0-255)
# print(v10.shape)
# dimensions in order: timestep (cumulative month), [v10,u10 (empty)], lat, lon)
# print April,v10 measurements (all lats and lons):
# print(v10[4,0,:,:])
width = v10.shape[3]
height = v10.shape[2]

v_data = v10[timestep,0,:,:].data # .data to convert from masked array to np array
u_data = u10[timestep,0,:,:].data

vmax = v_data.max()
vmin = v_data.min()
umax = u_data.max()
umin = u_data.min()

# recode javascript from https://github.com/mapbox/webgl-wind/blob/master/data/prepare.js
# js code:
# for (let y = 0; y < height; y++):
#     for (let x = 0; x < width; x++):
#         const i = (y * width + x) * 4;
#         const k = y * width + (x + width / 2) % width
#         png.data[i + 0] = Math.floor(255 * (u_data.values[k] - u.minimum) / (u.maximum - u.minimum))
#         png.data[i + 1] = Math.floor(255 * (v.values[k] - v.minimum) / (v.maximum - v.minimum))
#         png.data[i + 2] = 0
#         png.data[i + 3] = 255
    
# populate empty array of pixels using RBGA format
# rescale values from 0 - 255
u_data = u_data.T
v_data = v_data.T

png_array = np.zeros((width,height, 4), dtype='int')
png_array[:,:,0] = np.floor_divide((255 * (u_data - umin) / (umax- umin)),1).astype(np.uint8) #R
png_array[:,:,1] = np.floor_divide((255 * (v_data - vmin) / (vmax- vmin)),1).astype(np.uint8) #G
png_array[:,:,2] = 0 #B
png_array[:,:,3] = 255 #A = alpha
#print(png_array[:,:,:])

#from PIL import Image
# fileName = "image.png"
# image = Image("RGBA",(width,height))
# image.putdata(png_array)
# image.save(fileName)

image_2d = np.hstack(map(np.uint8, png_array))
#image_2d = np.hstack(map(np.uint16, image_2d))
# img = Image.fromarray(png_array, 'RGB')            
# img.save('my.png')

# https://stackoverflow.com/questions/26534148/python-pillow-v2-6-0-paletted-png-256-how-to-add-an-alpha-channel
# img = Image.open("my.png")
# if img.mode == "RGB":
#     a_channel = Image.new('RGBA', img.size, 255)
#     img.putalpha(a_channel)

# help(png)
info_dict = {
  "height": height,
  "width": width
}

# write accompanying json used for metadata and interpolation scaling
png.from_array(image_2d, 'RGBA', info=info_dict).save("..//salish-sea-demo//wind//" + str(datefilename) + ".png")

data = {'source': 'https://www.ecmwf.int/en/forecasts/datasets/browse-reanalysis-datasets',
        'date': datefilename,
        'width': width,
        'height':height,
        'uMin': umin,
        'uMax': umax,
        'vMin': vmin,
        'vMax': vmax
       }

with open("..//salish-sea-demo//wind//" + str(datefilename) + ".json", 'w') as outfile:
    json.dump(data, outfile)

# image_2d = np.vstack(map(np.uint16, png_array))
# png.from_array(image_2d, 'RGBA').save("file.png")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:67: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


In [271]:



# data['people'] = []
# data['people'].append({
#     'name': 'Scott',
#     'website': 'stackabuse.com',
#     'from': 'Nebraska'
# })
# data['people'].append({
#     'name': 'Larry',
#     'website': 'google.com',
#     'from': 'Michigan'
# })
# data['people'].append({
#     'name': 'Tim',
#     'website': 'apple.com',
#     'from': 'Alabama'
# })



In [258]:
u_data[24]

array([-0.00295261,  0.19032019,  0.3499463 ,  0.38222364,  0.2533099 ,
        0.42428199,  0.56004244,  0.77189915,  1.01896751,  1.19287391,
        0.98375587,  0.73238387,  0.54243662,  0.54674026,  0.7278846 ,
        0.8110232 ,  1.13027543,  1.33626354,  1.45735247])

In [221]:
img = Image.open("file3.png")
img

In [250]:
u_data.T.shape

(25, 19)

In [ ]:
# to do - export json file of max min width etc

In [247]:
image_2d.shape

(25, 76)

In [248]:
image_2d.T

array([[ 57,  54,  51, ...,  68,  59,  52],
       [118, 120, 125, ..., 141, 140, 137],
       [  0,   0,   0, ...,   0,   0,   0],
       ...,
       [  0,   2,   6, ..., 104,  81,  70],
       [  0,   0,   0, ...,   0,   0,   0],
       [255, 255, 255, ..., 255, 255, 255]], dtype=uint8)

In [ ]:
import PIL
#https://stackoverflow.com/questions/54703674/how-do-i-make-my-numpy-image-take-an-alpha-channel-value-in-python



# Make into Numpy array of RGB and get dimensions
RGB = np.array(im)
h, w = RGB.shape[:2]

# Add an alpha channel, fully opaque (255)
RGBA = np.dstack((RGB, np.zeros((h,w),dtype=np.uint8)+255))

# Make mask of black pixels - mask is True where image is black
mBlack = (RGBA[:, :, 0:3] == [0,0,0]).all(2)

# Make all pixels matched by mask into transparent ones
RGBA[mBlack] = (0,0,0,0)

# Convert Numnpy array back to PIL Image and save
Image.fromarray(RGBA).save('result.png')

In [284]:
img = Image.open("..//salish-sea-demo//wind//2000-05.png")
img

In [287]:
pix = np.array(img)

In [293]:
pix[0]

array([[ 83, 101,   0, 255],
       [ 83, 105,   0, 255],
       [ 85, 111,   0, 255],
       [ 91, 114,   0, 255],
       [103, 123,   0, 255],
       [112, 128,   0, 255],
       [118, 137,   0, 255],
       [123, 140,   0, 255],
       [133, 136,   0, 255],
       [139, 132,   0, 255],
       [143, 129,   0, 255],
       [145, 128,   0, 255],
       [146, 129,   0, 255],
       [146, 130,   0, 255],
       [144, 122,   0, 255],
       [145, 117,   0, 255],
       [139, 106,   0, 255],
       [134, 106,   0, 255],
       [118, 112,   0, 255],
       [100, 118,   0, 255],
       [ 86, 123,   0, 255],
       [ 74, 127,   0, 255],
       [ 61, 129,   0, 255],
       [ 49, 126,   0, 255],
       [ 46, 118,   0, 255]], dtype=uint8)